Please run the next 2 cells before using the notebook. Because the HBP's Collaboratory does not provide an implementation of the MPI software, widely used in parallel computing, it has to be built by this notebook and the process can take up to 30 minutes.

The output provided by the cell below is to follow progress, but can be minimized by clicking or double clicking the empty space in the margin left of the output.

In [ ]:
mpi_test = !mpiexec -n 2 python -c "print('OK')"
mpi_ok = False
try:
    mpi_ok = len(mpi_test) == 2 and mpi_test[1] == 'OK'
except:
    pass
if not mpi_ok:
    !wget https://download.open-mpi.org/release/open-mpi/v4.0/openmpi-4.0.3.tar.gz
    !gunzip -c openmpi-4.0.3.tar.gz | tar xf -
    !cd openmpi-4.0.3 && ./configure --prefix=/home/jovyan
    !cd openmpi-4.0.3 && make all install
else:
    print("MPI OK!")

In [ ]:
_ = !pip3 install hbp_service_client --upgrade
import os
from IPython.core.display import HTML
print("Downloading files...")
client = get_hbp_service_client()
# Download the storage contents
for file in client.storage.list("/79605"):
    if not os.path.exists(file):
        print("Downloading '" + file + "'")
        client.storage.download_file(os.path.join("/79605", file), file)
# Initialize
from notebook_tools import init, finalize
init()
# Run installation script
_ = !chmod +x installation.sh
_ = !./installation.sh
# Finalize
finalize()

# Cerebellar scaffold with multicompartmental simulations

This use-case demonstrates a simulation of the cerebellar cortex using multicompartmental neuron models. The network is constructed using our scaffold approach circuit builder.

## Notebook structure

1. Code setup
    1. [Initialisation of dependencies](#Initialisation-of-dependencies)
    2. [Network construction](#Network-construction)
    3. [Network simulation](#Network-simulation)
2. Analysis
    1. [Voltage recordings](#Voltage-recordings)
    2. [Spike raster](#Spike-raster)

## Code setup

### Initialisation of dependencies

This notebook requires our `dbbs-scaffold`, `dbbs-models` and `pynb-sim` Python packages, and the external dependencies [Rtree](https://github.com/Toblerity/rtree), [OpenMPI](https://www.open-mpi.org/) and [NEURON](https://neuron.yale.edu/neuron/).

<span style="color: red">This cell needs to be executed after each kernel restart</span>, whenever an `undefined symbol: Error_GetLastErrorNum` error occurs or if the simulation finishes immediately.

In [ ]:
%%capture --no-display
!ln -sfn /home/jovyan/.local/nrn-7.6/ /home/jovyan/.local/nrn
import os, sys
from notebook_tools import init, finalize_packages
init()
!{sys.executable} -m pip install rtree mpi4py
!{sys.executable} -m pip install pynb-sim==0.2.0 dbbs-scaffold==3.2.8rc0 dbbs_models dbbs_mod_collection
import rtree, scaffold, dbbs_models
finalize_packages()

### Network construction

The `dbbs-scaffold` package can reconstruct cerebellar networks based on a configuration file. For multicompartmental models it also requires an extra HDF5 file containing processed morphologies. For this notebook we provide `morphologies.hdf5` containing a granule cell, Golgi cell, Purkinje cell, stellate cell and basket cell. 

We're going to create a new configuration file based on the template `neuron.json`:

In [ ]:
!scaffold make-config my_configuration.json -t=neuron.json

And compile a network from it (this may take up to 10 minutes):

In [ ]:
!scaffold -c=my_configuration.json compile

# Comment the command above, and uncomment the command below
# if you would like to see progress output:

# !scaffold -v=3 -c=my_configuration.json compile

#### Full network plot (only cell soma)

<span style="color: darkred">Attention!</span> These are heavy plots and might take long to load or cause the notebook to become unresponsive. <span style="color: red">DO NOT</span> save the notebook with these outputs present! When opening the notebook all plots will be loaded simultaneously and will surely cause the notebook to crash.

In [ ]:
from scaffold.core import from_hdf5
from scaffold.plotting import plot_network
scaffold = from_hdf5("neuron.hdf5")
plot_network(scaffold, from_memory=False)

#### Granular layer neurons

<span style="color: darkred">Attention!</span> These are heavy plots and might take long to load or cause the notebook to become unresponsive. <span style="color: red">DO NOT</span> save the notebook with these outputs present! When opening the notebook all plots will be loaded simultaneously and will surely cause the notebook to crash.

In [ ]:
from scaffold.core import from_hdf5
from scaffold.plotting import plot_detailed_network
import numpy as np
scaffold = from_hdf5("neuron.hdf5")
ids = np.array([scaffold.get_placement_set("golgi_cell").identifiers[0], *scaffold.get_placement_set("granule_cell").identifiers[0:5]])
plot_detailed_network(scaffold, ids=ids)

#### Molecular layer neurons

<span style="color: darkred">Attention!</span> These are heavy plots and might take long to load or cause the notebook to become unresponsive. <span style="color: red">DO NOT</span> save the notebook with these outputs present! When opening the notebook all plots will be loaded simultaneously and will surely cause the notebook to crash.

In [ ]:
from scaffold.core import from_hdf5
from scaffold.plotting import plot_detailed_network
import numpy as np
scaffold = from_hdf5("neuron.hdf5")
ids = np.array([scaffold.get_placement_set("purkinje_cell").identifiers[0], scaffold.get_placement_set("stellate_cell").identifiers[0]])
plot_detailed_network(scaffold, ids=ids)

#### Configuring the input

The configuration supports many features and a full explanation can be found on the [public documentation](https://dbbs-scaffold.readthedocs.io/en/latest/). For this notebook we provide a small interface that can set the input stimulation to the mossy fibers of the network.

The mossy fibers terminate in the cerebellar cortex as _glomeruli_, structures where synaptic contacts are made between the glomerulus, granule cell and Golgi cell.

We will simulate the arrival of action potentials at the glomerulus at noisy periodic intervals:

In [ ]:
import notebook_tools
notebook_tools.stimulation_interface("ec7ed747-f396-4d23-b075-650ad1e1593a")

_start_ is the time in milliseconds when the stimulation begins. _number_ is the fixed amount of action potentials that will be simulated. _interval_ is the __average__ interval between 2 spikes. The signal is noisy so the actual action potential intervals will differ.

### Network simulation

After construction `neuron.hdf5` now contains all the information required to run the simulation. On your own machine you can run a simulation using this command:

    mpiexec -n 4 scaffold -v=3 simulate poc --hdf5=neuron.hdf5

*The simulation takes more than an hour to complete. Instead of waiting for the results, the plots below the next code cell can be previewed with results of the default simulation.*

In this notebook we're going to use `pynbsim` to track its progress:

In [ ]:
import pynbsim, uuid
# Monitor a progress file
reader = pynbsim.ProgressFileReader("progress.io")
# Create a unique token for this simulation
simulation_token = uuid.uuid4()
# Display the progress in a label below
pynbsim.progress_text("ea32d4cc-63c5-4586-9a4e-1c80d34bddbf", simulation=simulation_token)
# Start the simulation
pynbsim.simulate(
    token=simulation_token,
    # Start MPI with 4 nodes and run the scaffold: 
    # - Make it report to the progress file
    # - Simulate the 'poc' simulation using the `neuron.hdf5` compiled network
    command="mpiexec -n 1 scaffold -r=progress.io -v=3 simulate poc --hdf5=neuron.hdf5", 
    progress_reader=reader
)

If the simulation isn't working, try running the command below to find out what the error is:

In [ ]:
!mpiexec -n 1 scaffold -r=progress.io -v=3 simulate poc --hdf5=neuron.hdf5

## Analysis

The simulation outputs its recordings into an HDF5 file with 2 categories of `recorders`: `spike_recorder`s and `voltage_recorder`s. In the configuration file spike and voltage recorders have been requested for each cell at the soma. Additionally voltage recorders have been placed on a random dendrite of a representative of each cell type.


### Spike raster

Shows the activity of all cells during the entire simulation.

In [ ]:
from scaffold.plotting import hdf5_plot_spike_raster
from scaffold.output import latest_results
import notebook_tools
from h5py import File

# Open the last generated HDF5 file of the `poc` simulation
with File(latest_results("poc"), "r") as f:
    # Plot the spike raster
    hdf5_plot_spike_raster(f["/recorders/soma_spikes"], input_region=notebook_tools.get_stimulation_region())

\* *The duration of input stimulation is marked in grey. This data is taken from the currently configured stimulation and not necesarily the stimulation that was used for the voltage traces. If you suspect a difference, reconfigure the input stimulation and run the simulation again. Do not change the configured stimulation afterwards.*

### Voltage recordings

Shows the membrane potential at the soma and a random dendritic branch for 1 random neuron of each population.

In [ ]:
from scaffold.plotting import hdf5_gather_voltage_traces, plot_traces, CellTraceCollection
from scaffold.output import latest_results
from h5py import File
import notebook_tools

# Open the last generated HDF5 file of the `poc` simulation
with File(latest_results("poc"), "r") as f:
    # Collect traces from cells across multiple recording groups.
    cell_traces = hdf5_gather_voltage_traces(f, "/recorders/", ["soma_voltages", "dendrites"])
    # Take only those cells that on top of their soma also had a dendrite recorded
    representatives = CellTraceCollection([c for c in cell_traces if len(c.traces) > 1])
    representatives.set_legends(["Soma (mV)", "Dendrite (mV)"])
    representatives.set_colors(["Crimson", "Blue"])
    plot_traces(representatives, input_region=notebook_tools.get_stimulation_region())

\* *The duration of input stimulation is marked in grey. This data is taken from the currently configured stimulation and not necesarily the stimulation that was used for the voltage traces. If you suspect a difference, reconfigure the input stimulation and run the simulation again. Do not change the configured stimulation afterwards.*

### Stimulation boxplot

This plot shows the mean firing rate of the different cell types during stimulation.

In [ ]:
import notebook_tools
notebook_tools.stimulation_boxplot()